In [1]:
import numpy as np
import os
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from skimage import color, transform
from pathlib import Path
import keras
from PIL import Image
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin

2025-10-25 12:30:39.442008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 12:30:39.601265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 12:30:42.528639: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Loading Saved Model

In [2]:
curr_dir = os.getcwd()
curr_dir

'/home/root123/GitHub/Two_Months_ML_Journey/Week 4'

In [3]:
cnn = keras.saving.load_model(f'{curr_dir}/saved_models/cnn_scratch_model.keras')

I0000 00:00:1761374744.561317   40502 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


## Creating Pipeline

In [4]:
# # creating a function transformer to pass it to pipeline

# def preprocess(img):
#     # Convert PIL Image → NumPy array
#     if isinstance(img, Image.Image):
#         img = np.array(img)
#     # grayscale (no channel dimension)
#     if img.ndim == 2:
#         img = color.gray2rgb(img)
#         print('Gray to RGB')
#     # RGBA (4 channels)
#     elif img.shape[2] == 4:
#         img = color.rgba2rgb(img)
#         print('RGBA to RGB')

#     # Resize to 128x128
#     resized_img = transform.resize(img, (128, 128), anti_aliasing=True) # anti_aliasing makes img smoother when shrinked
#     resized_img = resized_img.astype('float32')
#     norm_resized_img = resized_img / 255.0
#     return norm_resized_img

# # converting our function to 'Function Transformer' for compatibility with Sklearn's pipeline
# rgb_resize_transformer = FunctionTransformer(
#     lambda imgs: np.array([preprocess(img) for img in imgs])
# )

In [5]:
class KerasPreprocessorWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, target_size=(128, 128)):
        self.target_size = target_size

    def fit(self, X, y=None):
        # no learning from data here, but required by sklearn's API
        return self

    def transform(self, X):
        
            # Convert PIL images to numpy arrays if needed
        if isinstance(X, list):
            X = [np.array(img) if hasattr(img, 'size') else img for img in X]
            X = np.stack(X, axis=0)  # shape: (batch, H, W, C)
        elif isinstance(X, Image.Image):  # single image
            X = np.expand_dims(np.array(X), axis=0)
        # Ensure the input is a TensorFlow tensor
        X = tf.convert_to_tensor(X)

        # Normalize dtype and scale to [0,1]
        X = tf.image.convert_image_dtype(X, tf.float32)

        # Handle grayscale (1 channel) or RGBA (4 channels)
        num_channels = tf.shape(X)[-1]

        def to_rgb_if_grayscale():
            return tf.image.grayscale_to_rgb(X)

        def to_rgb_if_rgba():
            return X[..., :3]

        def identity():
            return X

        X = tf.case(
            [
                (tf.equal(num_channels, 1), to_rgb_if_grayscale),
                (tf.equal(num_channels, 4), to_rgb_if_rgba)
            ],
            default=identity,
            exclusive=True
        )

        # Resize to target size
        X = tf.image.resize(X, self.target_size)

        # Return a TensorFlow tensor (not NumPy)
        return X

In [6]:
pipeline = Pipeline(steps=[
    ('preprocess', KerasPreprocessorWrapper()),
    ('model', cnn),
])

In [7]:
destination_path = 'test_data'
predict_img_path = curr_dir/Path(destination_path)
predictions = [
    'Pepper Bell Bacterial Spot',
    'Pepper Bell Healthy',
    'Potato Early Blight',
    'Potato Healthy',
    'Potato Late Blight',
    'Tomato Target Spot',
    'Tomato Mosaic Virus',
    'Tomato Yellow Leaf Curl Virus',
    'Tomato Bacterial Spot',
    'Tomato Early Blight',
    'Tomato Healthy',
    'Tomato Late Blight',
    'Tomato Leaf Mold',
    'Tomato Septoria Leaf Spot',
    'Tomato Spider Mites',
]   


In [9]:
for img_file in predict_img_path.iterdir():
    if img_file.suffix.lower() in ['.jpg', '.png']:
        # Open image
        img = Image.open(img_file)
        
        # Run through your pipeline (wrap in list or array)
        pred = pipeline.predict([img])
        print(pred[0])
        print(f'Prediction: {predictions[np.argmax(pred)]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[2.9700295e-06 2.1409370e-09 9.1675460e-01 7.7063903e-02 4.7425615e-09
 1.1051750e-17 7.5241063e-05 6.1033675e-03 2.8543194e-13 2.8005352e-11
 3.3696212e-15 3.1860172e-12 3.9067681e-17 5.8525017e-26 1.0914473e-10]
Prediction: Potato Early Blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[2.22441016e-16 1.41301196e-13 2.41514961e-20 9.11008790e-13
 1.02097015e-13 8.18089116e-04 1.39774192e-08 6.76087138e-12
 3.21232704e-07 7.22444895e-07 1.81133203e-07 9.99180496e-01
 1.00546371e-09 1.71021902e-15 2.25368439e-07]
Prediction: Tomato Late Blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[9.99998927e-01 1.08403901e-06 3.90078899e-14 6.01822966e-11
 2.42632723e-14 1.99905483e-17 2.87295517e-14 1.28303694e-12
 1.22091205e-17 8.88851145e-12 7.85472886e-27 5.75513641e-18
 2.33248550e-22 5.34651416e-41 6.67183516e-22]
Prediction: Pepper Bell Bacterial Spot
